In [24]:
import os
from pathlib import Path
import cv2
import torch
from PIL import Image
from tqdm import tqdm



In [25]:
video_path = 'inputs/media/Clips/'
output_txt_path = 'outputs/yolo5_classification/'
custom_model = True
# custom_model = True

def get_video_paths():
    """
    Recursively Returns a list of all the videos in the directory
    """
    return [str(p) for p in Path.rglob(Path(video_path), '*.mp4')]


In [26]:
model = None
if custom_model:
    output_txt_path.replace('yolo5_classification', 'yolo5_classification_Ours')
    custom_model_path = 'custom_train\\yolov5\\runs\\train\\exp38\weights\\last.pt'
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=custom_model_path, force_reload=True)
else:
    model = torch.hub.load('ultralytics/yolov5', 'yolov5m')


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\ethan/.cache\torch\hub\master.zip
YOLOv5  2022-1-11 torch 1.10.1 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
Model Summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [27]:
def classify_image(img):
    results = model([img])
    return results


def classify_images(images):
    results = model(images)
    return results

In [28]:
def convert_to_yolo(x1, y1, x2, y2,sizes):
    x_center = (x1 + x2) / 2
    y_center = (y1 + y2) / 2
    width = x2 - x1
    height = y2 - y1

    img_width, img_height = sizes
    # normalize
    x_center = x_center / img_width
    y_center = y_center / img_height
    width = width / img_width
    height = height / img_height


    return x_center, y_center, width, height

def convert_to_minmax(x_center, y_center, width, height, sizes):
    img_width, img_height = sizes
    # normalize
    x_center = x_center * img_width
    y_center = y_center * img_height
    width = width * img_width
    height = height * img_height

    x1 = x_center - width/2
    y1 = y_center - height/2
    x2 = x_center + width/2
    y2 = y_center + height/2

    # round
    x1 = round(x1)
    y1 = round(y1)
    x2 = round(x2)
    y2 = round(y2)

    return x1, y1, x2, y2


In [29]:
classes = ['car', 'motorbike', 'truck', 'priority', 'bus']
coco_names = []
with open('inputs/weights/coco.names', 'r') as f:
    for line in f:
        coco_names.append(line.strip().lower())

coco_ours_map = {}
for c in classes:
    if c in coco_names:
        coco_ours_map[c] = classes.index(c)


def append_box_to_file(path, frame_no, results, sizes):
    results = results.pandas().xyxy[0]
    str_to_append = ""
    for i, row in results.iterrows():
        x1, y1, x2, y2 = row['xmin'], row['ymin'], row['xmax'], row['ymax']


        x_center, y_center, width, height = convert_to_yolo(x1, y1, x2, y2, sizes)
        

        conf, clas_id, name = row['confidence'], row['class'], row['name']

        if name not in coco_ours_map:
            continue

        our_class_id = coco_ours_map[name]
        # str_to_append  += str(our_class_id) + ' ' + str(x1) + ' ' + str(y1) + ' ' + str(x2) + ' ' + str(y2) + '\n'
        str_to_append += str(our_class_id) + ' ' + str(x_center) + ' ' + str(y_center) + ' ' + str(width) + ' ' + str(height) + '\n'

    # print('Saving Frame: ', frame_no)
    new_path=os.path.join(path, str(frame_no)+'.txt')

    # create if not exists
    if not os.path.exists(new_path):
        # if subdirs do not exist, create aswell
        if not os.path.exists(os.path.dirname(new_path)):
            os.makedirs(os.path.dirname(new_path))

    with open(new_path, 'w') as f:
        f.write(str_to_append)


In [30]:
def draw_image(img, results):
    results = results.pandas().xyxy[0]
    for i, row in results.iterrows():
        x1, y1, x2, y2 = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        x1 = round(x1)
        y1 = round(y1)
        x2 = round(x2)
        y2 = round(y2)

        size = img.shape[:2]
        x1, y1, x2, y2 = convert_to_minmax(*convert_to_yolo(*(x1, y1, x2, y2), size),size)
        
        conf,clas_id,name = row['confidence'], row['class'], row['name']
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(img, name, (x1, y1),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    return img


In [46]:
def test(frame, coords, model_start_from_centre=True):
    x_start = None
    for i, row in coords.iterrows():
        x_start, y_start, width, height = convert_to_yolo(row['xmin'], row['ymin'], row['xmax'], row['ymax'], frame.shape[:2])

    if not x_start:
        return

    x_res = frame.shape[0]
    y_res = frame.shape[1]

    if model_start_from_centre:
        x_start = x_start - width/2
        y_start = y_start - height/2
        
    # calculate the box's coordinates on the image
    x_start = round(x_start*x_res)
    y_start = round(y_start*y_res)
    width = round(width*x_res)
    height = round(height*y_res)

    

    x_end = x_start + width
    y_end = y_start + height
    cv2.rectangle(frame, (x_start, y_start),(x_end, y_end), (0, 255, 0), 2) #draws the box
    cv2.putText(frame, 'karozza', (x_start, y_start), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)    # label the category
    cv2.imshow('frame', frame)
    cv2.waitKey(1)


In [47]:
def label_videos(videos, show_video=False, save=False):
    for video_path in videos:
        total_frames = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT)
        video = cv2.VideoCapture(video_path)


        video_name = video_path.split('/')[-1]
        coord_path = os.path.join(output_txt_path, video_name)

        print('Labelling video: {}'.format(video_name))

        labelled_frames = []
        while True:
            _, frame = video.read()
            if frame is None:
                break

            frame_no = video.get(cv2.CAP_PROP_POS_FRAMES)
            print(f'Frame: {frame_no}/{total_frames}',end='\r')
            result = classify_image(frame)

            test(frame, result.pandas().xyxy[0], True)

            if save:
                size = frame.shape[:2]
                append_box_to_file(coord_path, frame_no, result, size)

            if show_video:
                img = draw_image(frame, result)
                cv2.imshow('frame', img)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        print('\n')
        video.release()
        cv2.destroyAllWindows()

In [48]:
videos = get_video_paths()
videos = [v for v in videos if 'original' in v.lower()]
videos = [v for v in videos if 'test' in v.lower()]
print('Found {} total videos'.format(len(videos)))

label_videos(videos, show_video=False,save=False)


Found 8 total videos
Labelling video: inputs\media\Clips\Original\Test\Day_1.mp4
